### Initialization
* Check whether the runtime is host or local.
* Mount Google Drive when using the host runtime.

In [0]:
try:
  from google.colab import drive
  drive.mount('/gdrive')
  runtime = "host"
except:
  runtime = "local"

### Parameters

In [0]:
#@title Parameters
#@markdown |Name            |Description|
#@markdown |:---            |:---|
#@markdown |`seed`|The random seed|
seed = 3984 #@param {type: "number"}

#@markdown ### `deep-coder` Repositories
#@markdown |Name            |Description|
#@markdown |:---            |:---|
#@markdown |`repository_url`|The URL of `deep-coder` git repository (enabled only in the host runtime)|
#@markdown |`branch_name`   |The branch name (enabled only in the host runtime)|
repository_url = "https://github.com/HiroakiMikami/deep-coder" #@param {type: "string"}
branch_name = "master" #@param {type: "string"}

#@markdown ### Settings
#@markdown |Name    |Description|
#@markdown |:---    |:---|
#@markdown |`device`|The id of GPU. `-1` means that CPU is used.|
device = 0 #@param {type: "number"}

#@markdown ### URLs
#@markdown |Name              |Description|
#@markdown |:---              |:---|
#@markdown |`model_shape_path`|The file path of the model shape.|
#@markdown |`model_path`      |The file of the model parameters.|
model_shape_path = "./examples/medium/trained-model/model-shape.pickle" #@param {type: "string"}
model_path = "./examples/medium/trained-model/model.npz" #@param {type: "string"}


### Setup
* Fix the random seed
* Download the codebase
  1. Clone git repository and move to the specified branch
  2. Initialize submodule
  3. Install chainer and cupy

In [0]:
import numpy as np
import random

SEED_MAX = 2**32 - 1

root_rng = np.random.RandomState(seed)
random.seed(root_rng.randint(SEED_MAX))
np.random.seed(root_rng.randint(SEED_MAX))

In [0]:
if runtime == "host":
  %cd /content
  !rm -rf deep-coder
  ![ ! -e deep-coder ] && git clone $repository_url deep-coder
  %cd deep-coder
  !git checkout origin/$branch_name
  !git submodule init
  !git submodule update
  !make -C DeepCoder_Utils/enumerative-search -j `nproc`
  !curl https://colab.chainer.org/install | sh -
  !pip install tqdm

### Examples

In [0]:
inputs = [[1, [-1, 3, 2, 4, -5]], [0, [-2, 2, 3, 3]], [2, [1, 2, 3]], [1, [0, 1, 2]]] #@param {type: "raw"}
outputs = [-1, -2, 3, 1] #@param {type: "raw"}

### Run Inference

In [0]:
import pickle
import os
import chainer as ch
from chainer import datasets
from src.dataset import EncodedDataset, Dataset
import src.inference as I
from src.model import ModelShapeParameters
from src.dataset import Example
from tqdm import tqdm_notebook as tqdm

with open(model_shape_path, "rb") as f:
  model_shape = pickle.load(f)

  model = I.InferenceModel(model_shape)
ch.serializers.load_npz(model_path, model.predictor)

pred = I.predict_with_neural_network(model_shape, model)


examples = [Example(inputs, output) for inputs, output in zip(inputs, outputs)]
print(examples)
result = I.search(
    os.path.join(os.getcwd(), "DeepCoder_Utils",
                 "enumerative-search", "search"),
    100,
    model_shape.dataset_metadata.value_range,
    examples,
    2,
    pred)

if result.is_solved:
    print("Time: {} sec".format(result.time_seconds))
    print("#ExploredNodes: {}".format(result.explored_nodes))
    print(result.solution)
else:
    print("Failed to synthesize")
